Bước 1: Khởi tạo môi trường và dữ liệu nền tảng

In [7]:
# Cấu trúc dữ liệu cho menu
PIZZA_MENU = {
"hải sản": {"nhỏ": 150, "vừa": 200, "lớn": 250},
"thập cẩm": {"nhỏ": 140, "vừa": 190, "lớn": 240}
}
# Từ đồng nghĩa cho từng lựa chọn
SYNONYMS = {
"greeting": ["hi", "hello", "xin chào", "chào", "hey"],
"hải sản": ["pizza hải sản", "hải sản", "1"],
"thập cẩm": ["pizza thập cẩm", "thập cẩm", "2"],
"nhỏ": ["nhỏ", "small", "1"],
"vừa": ["vừa", "medium", "2"],
"lớn": ["lớn", "large", "3"],
"có": ["có", "yes", "ok"],
"không": ["không", "ko", "no"]
}


Bước 2: Xử lý và chuẩn hóa đầu vào

In [8]:
def normalize_input(text):
    text = text.lower().strip()
    for key, values in SYNONYMS.items():
        if text in values:
            return key
    return text
print(normalize_input("medium"))


vừa


Bước 3: Xây dựng các phản hồi hội thoại

In [9]:
# Hàm này giúp xây dựng tin nhắn chào hỏi khách hàng
def get_greeting():
    """Tạo tin nhắn chào mừng và thực đơn."""
    menu = "\n".join([f" {i}. Pizza {n.title()}" for i, n in enumerate(PIZZA_MENU.keys(), 1)])
    return f"Pizza AIVN xin chào!\n\nThực đơn hôm nay:\n{menu}\n\nQuý khách chọn loại nào ạ?"

# Hàm này giúp lấy thông tin về kích thước bánh
def get_size_msg():
    """Tạo tin nhắn chọn size."""
    return "Tuyệt vời! Pizza AIVN rất vui khi quý khách đã chọn được món yêu thích!\n\n**AIVN xin mời quý khách chọn kích cỡ bánh phù hợp:**\n\n **Nhỏ** - Dành cho 1-2 người\n\n **Vừa** - Dành cho 2-3 người\n\n**Lớn** - Dành cho 3-4 người\n\n Quý khách muốn chọn cỡ nào ạ?"

# Hàm này giúp xác nhận đơn hàng
def get_confirm_msg(pizza, size, price):
    """Tạo tin nhắn xác nhận đơn hàng."""
    return f"**Đặt hàng thành công!**\n\n Pizza AIVN xin xác nhận đơn hàng của quý khách:\n\n **Loại bánh:** Pizza {pizza.title()}\n **Kích cỡ:** {size.title()}\n **Giá tiền:** {price}.000 VNĐ\n\n Quý khách có muốn đặt thêm pizza nào khác không ạ? (Có/Không)"

def get_thanks_msg(orders):
    """Tạo tin nhắn cảm ơn và tổng kết đơn hàng."""
    if not orders:
        return "Pizza AIVN xin chân thành cảm ơn quý khách!"
    
    # Xây dựng chuỗi tổng kết đơn hàng
    summary = "\n".join([
        f" {i}. Pizza {o['type'].title()} - {o['size'].title()} - {o['price']}.000đ"
        for i, o in enumerate(orders, 1)
    ])
    
    total = sum(o['price'] for o in orders)
    
    return (
        f"**Pizza AIVN xin chân thành cảm ơn quý khách!**\n\n"
        f"Đơn hàng của quý khách sẽ được chuẩn bị ngay!\n\n"
        f"**Chi tiết đơn hàng:**\n{summary}\n\n"
        f"**Tổng thanh toán: {total}.000 VNĐ** \n\n"
        f"Pizza sẽ được giao đến tay quý khách trong thời gian sớm nhất!\n\n"
        f"---\n Quý khách muốn đặt thêm? Hãy nói 'Hi' để bắt đầu đơn hàng mới nhé!"
    )


Bước 4: Xây dựng Logic xử lý chính

In [10]:
def process_input(text, state):
    """
    Xử lý input của người dùng dựa trên trạng thái hiện tại.
    Trả về (response, new_state).
    """
    normalized = normalize_input(text)
    current_state = state["current_state"]
    order = state["current_order"]
    orders = state["all_orders"]
    
    # Phản hồi mặc định nếu không khớp quy tắc
    response = "Xin lỗi quý khách, Pizza AIVN không hiểu yêu cầu của quý khách. Vui lòng thử lại hoặc nói 'Hi' để bắt đầu lại ạ!"
    
    # Cho phép thoát bất cứ lúc nào
    if normalized == "quit":
        state["current_state"] = "quit"
        return "Cảm ơn quý khách đã sử dụng dịch vụ. Tạm biệt!", state
    
    # Quy trình xử lý dựa trên trạng thái (State Machine)
    if current_state == "wait":
        if normalized == "greeting":
            state["current_state"] = "menu"
            state["current_order"] = {}  # Bắt đầu đơn hàng mới
            response = get_greeting()
            
    elif current_state == "menu":
        if normalized in PIZZA_MENU:
            order["type"] = normalized
            state["current_state"] = "size"
            response = get_size_msg()
            
    elif current_state == "size":
        if normalized in ["nhỏ", "vừa", "lớn"]:
            order["size"] = normalized
            order["price"] = PIZZA_MENU[order["type"]][normalized]
            state["current_state"] = "confirm"
            response = get_confirm_msg(order["type"], normalized, order["price"])
            
    elif current_state == "confirm":
        orders.append(order.copy())  # Lưu đơn hàng vào giỏ
        state["all_orders"] = orders
        
        if normalized == "có":
            state["current_state"] = "menu"
            state["current_order"] = {}  # Reset cho đơn mới
            response = "**Tuyệt vời!** Pizza AIVN rất vui khi được phục vụ quý khách thêm!\n\n" + get_greeting()
        else:  # Nếu người dùng "không"
            state["current_state"] = "wait"  # Chuyển về trạng thái chờ
            response = get_thanks_msg(orders)
            state["all_orders"] = []  # Xóa giỏ hàng sau khi cảm ơn
    
    # Cập nhật lại trạng thái
    state["current_order"] = order
    return response, state

Bước 5: Xây dựng vòng lặp hội thoại chính

In [13]:
def main():
    """Hàm chính chạy ứng dụng chatbot trên Terminal"""
    # Khởi tạo trạng thái ban đầu
    # 'wait': Chờ lời chào
    # 'menu': Chờ chọn loại pizza
    # 'size': Chờ chọn kích cỡ
    # 'confirm': Chờ xác nhận đặt thêm
    state = {
        "current_state": "wait",
        "current_order": {},
        "all_orders": []
    }
    
    # In lời chào đầu tiên
    print("=" * 50)
    print("Chào mừng đến với AIVN Pizza Chatbot!")
    print("Hãy nói 'Hi' hoặc 'Xin chào' để bắt đầu.")
    print("Gõ 'quit' hoặc 'tạm biệt' bất cứ lúc nào để thoát.")
    print("=" * 50)
    
    # Bắt đầu vòng lặp hội thoại
    while True:
        # 1. Nhận input từ người dùng
        user_input = input("Quý khách: ")
        
        # 2. Xử lý input
        response, state = process_input(user_input, state)
        
        # 3. In phản hồi của bot
        print(f"\nChatbot: {response}\n")
        print("-" * 50)
        
        # 4. Kiểm tra điều kiện thoát
        if state["current_state"] == "quit":
            break

if __name__ == "__main__":
    main()

Chào mừng đến với AIVN Pizza Chatbot!
Hãy nói 'Hi' hoặc 'Xin chào' để bắt đầu.
Gõ 'quit' hoặc 'tạm biệt' bất cứ lúc nào để thoát.

Chatbot: Cảm ơn quý khách đã sử dụng dịch vụ. Tạm biệt!

--------------------------------------------------
